In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [3]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [4]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [5]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1040.0


In [6]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  276.0


In [7]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [8]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2817 personas en los últimos 7 días 

Un positivo PCR cada 2113 personas en los últimos 14 días


In [9]:
filtro = ((df['Medida'] == 'Confirmados PCR')| (df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [10]:
# Reorder columns
tabla = tabla[['Confirmados PCR', 'Confirmados PCR 7 días', 'Confirmados PCR 14 días', 'Población']]

In [11]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [12]:
tabla['Diferencia PCR'] = tabla['Confirmados PCR'] - dfAnterior['Confirmados PCR']

In [13]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PCR'] > 0].sort_values('Diferencia PCR', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Huelva,1040.0,174.0,307.0,510743.0,34.068015,60.108509,40.0
Huelva-Costa,560.0,118.0,174.0,288115.0,40.955868,60.392552,32.0
Huelva (capital),276.0,51.0,68.0,143663.0,35.499746,47.332995,19.0
Aljaraque,63.0,17.0,20.0,21260.0,79.962371,94.073377,7.0
Ayamonte,31.0,15.0,19.0,20946.0,71.612718,90.709443,5.0
Condado-Campiña,306.0,37.0,100.0,155057.0,23.862193,64.492412,5.0
Bonares,3.0,2.0,2.0,6058.0,33.014196,33.014196,1.0
Cabezas Rubias,8.0,6.0,7.0,730.0,821.917808,958.904110,1.0
Palma del Condado (La),32.0,2.0,11.0,10761.0,18.585633,102.220983,1.0


In [14]:
if tabla[tabla['Diferencia PCR'] > 0]['Diferencia PCR'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [15]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Cabezas Rubias,8.0,6.0,7.0,730.0,821.917808,958.904110,1.0
Villalba del Alcor,38.0,7.0,24.0,3338.0,209.706411,718.993409,1.0
Zalamea la Real,19.0,5.0,6.0,3068.0,162.972621,195.567145,0.0
Bollullos Par del Condado,35.0,7.0,22.0,14272.0,49.047085,154.147982,0.0
Isla Cristina,52.0,15.0,32.0,21264.0,70.541761,150.489090,0.0
Palos de la Frontera,21.0,12.0,13.0,11289.0,106.298166,115.156347,1.0
Cortegana,9.0,1.0,5.0,4666.0,21.431633,107.158165,0.0
Palma del Condado (La),32.0,2.0,11.0,10761.0,18.585633,102.220983,1.0
Santa Bárbara de Casa,7.0,1.0,1.0,1035.0,96.618357,96.618357,0.0
